In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from tensorflow.python.feature_column import feature_column_v2 as fc

from sklearn.utils import shuffle
from six.moves import urllib
import matplotlib.pyplot as plt
%matplotlib inline 

tf.enable_eager_execution()

In [0]:
# from google.colab import files
# files.upload()
# # Let's make sure the kaggle.json file is present.
# !ls -lha kaggle.json
# # Next, install the Kaggle API client.
# !pip install -q kaggle
# # The Kaggle API client expects this file to be in ~/.kaggle,
# # so move it there.
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# # This permissions change avoids a warning on Kaggle tool startup.
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle competitions download -c avazu-ctr-prediction

In [0]:
# !curl -O https://drive.google.com/drive/folders/1yYawFXxbBN0CInVx66LDQgBhl1nkXgA0/avazu_top_users.gz

# source = "https://drive.google.com/drive/folders/1yYawFXxbBN0CInVx66LDQgBhl1nkXgA0/avazu_top_users.gz"
# dest = "avazu_top_users.gz"
# urllib.request.urlretrieve(source, dest)

# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
# !pip install PyDrive


# UPLOAD DATA 

In [0]:
from google.colab import files
uploaded = files.upload()

!ls -ltr

Saving avazu_top_users.gz to avazu_top_users.gz
total 109760
drwxr-xr-x 1 root root      4096 Dec  7 17:32 sample_data
-rw-r--r-- 1 root root 112382320 Dec 10 18:29 avazu_top_users.gz


In [0]:
dataframe = pd.read_csv('avazu_top_users.gz')

In [0]:
# dataframe.groupby('click').count()

,id,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,hr
click,,,,,,,,,,,,,,,,,,,,,
0.0,4209308,4209308,4209308,4209308,4209308,4209308,4209308,4209308,4209308,4209308,...,4209308,4209308,4209308,4209308,4209308,4209308,4209308,4209308,4209308,4209308
1.0,673961,673961,673961,673961,673961,673961,673961,673961,673961,673961,...,673961,673961,673961,673961,673961,673961,673961,673961,673961,673961


# DATA PREP

In [0]:
# Create new feature for HR component & convert label feature into float
dataframe['hr'] = dataframe['hour'].astype(str).str[6:].astype(np.int64)
dataframe['click'] = dataframe['click'].astype(float)

In [34]:
# Print features type and number of unique values
col = list(dataframe)

for i in range(len(col)):
    x = dataframe[col[i]].unique()
    print (col[i] + ' = ' + str(dataframe[col[i]].dtypes) + ' = ' + str(len(x)))

id = float64 = 4883269
click = float64 = 2
hour = int64 = 240
C1 = int64 = 7
banner_pos = int64 = 4
site_id = object = 2042
site_domain = object = 4915
site_category = object = 4
app_id = object = 3498
app_domain = object = 319
app_category = object = 28
device_id = object = 1000000
device_ip = object = 1074289
device_model = object = 6057
device_type = int64 = 4
device_conn_type = int64 = 4
C14 = int64 = 2236
C15 = int64 = 8
C16 = int64 = 9
C17 = int64 = 365
C18 = int64 = 4
C19 = int64 = 63
C20 = int64 = 165
C21 = int64 = 57
hr = int64 = 24


In [0]:
# Print Unique values
col = list(dataframe)
valid_col = []
for i in range(len(col)):
  if col[i] not in('id', 'hour', 'site_id', 'site_domain', 'app_id','device_id', 'device_ip', 'device_model', 'C14'):
    x = dataframe[col[i]].unique()
    print (col[i] + ' = ' + str(dataframe[col[i]].unique()))  

In [35]:
# Need to fix -1 values
# dataframe.groupby('C20').count()

a = dataframe.loc[dataframe['click'] == 0].shape[0]
b = dataframe.loc[dataframe['click'] == 1].shape[0]
c = (a/(a+b))*100
print (c)

86.19856903234289


# CREATE SAMPLE DATA

In [36]:
sample_df = shuffle(dataframe)
sample_df = sample_df.sample(100000)
print (sample_df.shape)

(100000, 25)


In [37]:
a = sample_df.loc[sample_df['click'] == 0].shape[0]
b = sample_df.loc[sample_df['click'] == 1].shape[0]
c = (a/(a+b))*100
print (c)

86.27


In [0]:
# total_records = 100000

# tot_records_0 = round(0.98 * total_records)
# tot_records_1 = total_records - tot_records_0

# dataframe_0 = dataframe.loc[dataframe['click'] == 0]
# shuffle(dataframe_0)
# sample_df_0 = dataframe_0.sample(tot_records_0)

# dataframe_1 = dataframe.loc[dataframe['click'] == 1]
# shuffle(dataframe_1)
# sample_df_1 = dataframe_1.sample(tot_records_1)

# sample_df = pd.concat([sample_df_0, sample_df_1])
# sample_df = shuffle(sample_df)

# print (sample_df_0.shape)
# print (sample_df_1.shape)
# print (sample_df.shape)

In [0]:
# sample_df.groupby('click').count()

In [39]:
# Got Valid feature List, exclude unwanted ones
col = list(dataframe)
valid_col = []
for i in range(len(col)):
#   if col[i] not in('id', 'hour','device_id', 'device_ip', 'C20', 'C15', 'C16', 'C19'):
  if col[i] not in('id', 'hour','device_id', 'device_ip', 'C20'):
    valid_col.append(col[i])

# Create test/val/train data
sample_col_df = sample_df[valid_col]

train, test = train_test_split(sample_col_df, test_size=0.2, random_state = 42)
train, val = train_test_split(train, test_size=0.2,random_state = 42)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

64000 train examples
16000 validation examples
20000 test examples


In [0]:
# train.groupby('click').count()

# CREATE DATASETS FROM DATAFRAME

In [0]:
def dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('click')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.repeat().batch(batch_size)
  return ds

In [0]:
# Used for Custom Estimator
batch_size = 256
train_ds = dataset(train, batch_size=batch_size)
val_ds   = dataset(val, batch_size=batch_size)
test_ds  = dataset(test, shuffle=False, batch_size=batch_size)

In [0]:
# Used for Predefined Estimator
batch_size = 256
def train_inpf():
  train_ds = dataset(train, batch_size=batch_size)
  return train_ds

def test_inpf():
  test_ds  = dataset(test, shuffle=False, batch_size=batch_size)
  return test_ds

# # Ignore

Let's take a look at what the train dataset returns:

In [0]:
print(train_ds.take(1))

<TakeDataset shapes: ({C1: (?,), banner_pos: (?,), site_id: (?,), site_domain: (?,), site_category: (?,), app_id: (?,), app_domain: (?,), app_category: (?,), device_model: (?,), device_type: (?,), device_conn_type: (?,), C14: (?,), C15: (?,), C16: (?,), C17: (?,), C18: (?,), C19: (?,), C21: (?,), hr: (?,)}, (?,)), types: ({C1: tf.int32, banner_pos: tf.int32, site_id: tf.string, site_domain: tf.string, site_category: tf.string, app_id: tf.string, app_domain: tf.string, app_category: tf.string, device_model: tf.string, device_type: tf.int32, device_conn_type: tf.int32, C14: tf.int32, C15: tf.int32, C16: tf.int32, C17: tf.int32, C18: tf.int32, C19: tf.int32, C21: tf.int32, hr: tf.int32}, tf.float32)>


In [0]:
for feature_batch, label_batch in train_ds.take(1):
  print('All features:', list(feature_batch.keys()))
  print('A batch of hr:', feature_batch['hr'])
  print('A batch of labels:', label_batch )

Let's create and demonstrate a few different kinds of feature columns.

In [0]:
example_batch = list(train_ds.take(1))[0][0]
example_batch

In [0]:
def demo(feature_column):
  feature_layer = fc.FeatureLayer([feature_column])
  print(feature_layer(example_batch).numpy())

In [0]:
hr = fc.numeric_column("hr")
print (hr)
demo(hr)

In [0]:
# Print features type and number of unique values
col = list(dataframe)

for i in range(len(col)):
    x = dataframe[col[i]].unique()
    print (col[i] + ' = ' + str(dataframe[col[i]].dtypes) + ' = ' + str(len(x)))

id = float64 = 4883269
click = float64 = 2
hour = int64 = 240
C1 = int64 = 7
banner_pos = int64 = 4
site_id = object = 2042
site_domain = object = 4915
site_category = object = 4
app_id = object = 3498
app_domain = object = 319
app_category = object = 28
device_id = object = 1000000
device_ip = object = 1074289
device_model = object = 6057
device_type = int64 = 4
device_conn_type = int64 = 4
C14 = int64 = 2236
C15 = int64 = 8
C16 = int64 = 9
C17 = int64 = 365
C18 = int64 = 4
C19 = int64 = 63
C20 = int64 = 165
C21 = int64 = 57
hr = int64 = 24


# DEFINE FEATURE COLUMNS

In [0]:
C1  = fc.numeric_column('C1')
C14 = fc.numeric_column('C14')
C17 = fc.numeric_column('C17')
C18 = fc.numeric_column('C18')
C21 = fc.numeric_column('C21')

C15 = fc.numeric_column('C15')
C16 = fc.numeric_column('C16')
C19 = fc.numeric_column('C19')
# C20 = fc.numeric_column('C20')

device_type      = fc.numeric_column('device_type')
device_conn_type = fc.numeric_column('device_conn_type')
hr               = fc.numeric_column('hr')
banner_pos       = fc.numeric_column('banner_pos')

site_category = fc.categorical_column_with_vocabulary_list('site_category', list(sample_col_df['site_category'].unique()))
app_category  = fc.categorical_column_with_vocabulary_list('app_category', list(sample_col_df['app_category'].unique()))

app_domain    = fc.categorical_column_with_vocabulary_list('app_domain', list(sample_col_df['app_domain'].unique()))

# app_id        = fc.categorical_column_with_vocabulary_list('app_id', list(sample_col_df['app_id'].unique()))
# device_model  = fc.categorical_column_with_vocabulary_list('device_model', list(sample_col_df['device_model'].unique()))
# site_domain   = fc.categorical_column_with_vocabulary_list('site_domain', list(sample_col_df['site_domain'].unique()))
# site_id       = fc.categorical_column_with_vocabulary_list('site_id', list(sample_col_df['site_id'].unique()))

app_id = fc.categorical_column_with_hash_bucket('app_id', hash_bucket_size=1000)
device_model = fc.categorical_column_with_hash_bucket('device_model', hash_bucket_size=1000)
site_domain = fc.categorical_column_with_hash_bucket('site_domain', hash_bucket_size=1000)
site_id = fc.categorical_column_with_hash_bucket('site_id', hash_bucket_size=1000)

In [0]:
all_columns = [
    C1, C14, C17, C18, C21, C15, C16, C19, device_type, device_conn_type, hr, banner_pos,   # Numneric Features  
    #C20,
    
    fc.indicator_column(site_category),                 # 4 distinct values
    fc.indicator_column(app_category),                  # 28 distinct values        

    fc.embedding_column(app_domain, dimension=5),       # 319 distinct values
    
    fc.embedding_column(app_id, dimension=8),           # 3498 distinct values
    fc.embedding_column(device_model, dimension=9),     # 6057 distinct values
    fc.embedding_column(site_domain, dimension=9),      # 4915 distinct values
    fc.embedding_column(site_id, dimension=7)           # 2042 distinct values
    
#     fc.embedding_column(site_category, dimension=8)
#     fc.embedding_column(app_category, dimension=8),    
]

# PREDEFINED ESTIMATOR

In [47]:
# classifier = tf.estimator.LinearClassifier(feature_columns=all_columns)
# classifier.train(train_inpf)
# result = classifier.evaluate(test_inpf)

estimator = tf.estimator.DNNClassifier(feature_columns=all_columns, hidden_units=[128, 128], dropout=0.2)
estimator.train(train_inpf, steps=100)

result = estimator.evaluate(test_inpf, steps = 10)

for key,value in sorted(result.items()):
  print('%s: %s' % (key, value))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpm08bulb3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc2fbf131d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph wa

In [0]:
# estimator.get_variable_value()

# CUSTOM ESTIMATOR

In [0]:
feature_layer = fc.FeatureLayer(all_columns)

In [0]:
model = tf.keras.Sequential([
  feature_layer,
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dropout(rate=0.5),  
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dropout(rate=0.5), 
  tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
#   tf.keras.layers.Dense(1, activation='softmax')
])

In [0]:
model.summary()

In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [53]:
batch_size = 1000
steps_per_epoch_train = len(train) // batch_size
steps_per_epoch_val = len(val) // batch_size
print (steps_per_epoch_train)
print (steps_per_epoch_val)

model.fit(train_ds, 
          steps_per_epoch=steps_per_epoch_train,
          validation_data=val_ds, 
          validation_steps=steps_per_epoch_val,
          epochs=5)

64
16
Epoch 1/5
64/64 [==============================] - 9s 146ms/step - loss: 5.2868 - acc: 0.6682 - val_loss: 2.2705 - val_acc: 0.8591
Epoch 2/5
64/64 [==============================] - 9s 136ms/step - loss: 2.2302 - acc: 0.8616 - val_loss: 2.1171 - val_acc: 0.8687
Epoch 3/5
64/64 [==============================] - 9s 136ms/step - loss: 2.2537 - acc: 0.8601 - val_loss: 2.2076 - val_acc: 0.8630
Epoch 4/5
64/64 [==============================] - 9s 145ms/step - loss: 2.1907 - acc: 0.8641 - val_loss: 2.2312 - val_acc: 0.8616
Epoch 5/5
64/64 [==============================] - 9s 136ms/step - loss: 2.2859 - acc: 0.8581 - val_loss: 2.2666 - val_acc: 0.8594


Finally, let's evaluate our model on the test data.

In [54]:
loss, accuracy = model.evaluate(test_ds, steps=len(test) // batch_size)
print("Accuracy", accuracy)
print("Loss", loss)

20/20 [==============================] - 1s 72ms/step
Accuracy 0.8658203125
Loss 2.162721025943756
